In [1]:
import pandas as pd
import numpy as np

import statsmodels.formula.api as smf
import statsmodels.api as sm

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold 

from tqdm.notebook import tqdm


In [2]:
# LM estimation parameters
LM_LIST = ['gpt2-small', 'neo-125m']
N_SAMPLES_LIST = [512]
N_TOKENS_LIST = [5]
TEMPERATURE = 1.0

# Cross-validation parameters
N_SEEDS = 100
N_FOLDS = 10

# Required data paths
DATA_FOLDER = "../../data"
RESULTS_FOLDER = "crossvalidation_results"

## Preliminaries

### Load estimates of responsive and anticipatory measures

In [3]:
# Load data
data = pd.read_csv(f'{DATA_FOLDER}/psyling/aligned_with_estimates_23july.csv')
data.columns = [col.replace('-', '_') for col in data.columns]

# Define names of measures and predicted variables
RATINGS = ['rating_mean', 'cloze_p_smoothed']
ERP = ['N400', 'P600', 'ELAN', 'LAN', 'N400', 'EPNP', 'P600', 'PNP']
RT = ['RTfirstfix', 'RTfirstpass', 'RTrightbound', 'self_paced_reading_time']
VARIABILITY = ['entropy'] 
ALL_PREDICTED_VARIABLES = RATINGS + RT + ERP + VARIABILITY

BASELINE_PREDICTORS = ['Subtlex_log10', 'context_length', 'length']
BASELINE_PREDICTORS_RT = [
    'Subtlex_log10', 'context_length', 'length', 
    'Subtlex_log10_prev', 'length_prev',
    'Subtlex_log10_prev_prev', 'length_prev_prev'
]

EXACT_METRICS = ['prob', 'surprisal', 'expected_prob', 'entropy']
SAMPLING_METRICS = ['decontextualised', 'expected_decontextualised', 'expected_seq_decontextualised', 'sequence_entropy']

ANTICIPATORY = ['expected_prob', 'expected_decontextualised', 'expected_seq_decontextualised', 'sequence_entropy']
RESPONSIVE = ['prob', 'decontextualised']



### Utility functions

In [4]:
def all_pairs(list):
    """
    Generate all pairs of elements in a list.
    """
    pairs = []
    for i in range(len(list)):
        for j in range(i+1, len(list)):
            pairs.append([list[i], list[j]])
    return pairs

def compute_rsquared(regressor, test_set, predicted_var):
    """
    Compute R-squared for a given regressor and test set.
    """
    y_pred = regressor.predict(test_set)
    y_true = test_set[predicted_var]
    residuals = y_true - y_pred
    rss = np.sum(residuals**2)
    tss = np.sum((y_true - np.mean(y_true))**2)
    rsquared = 1 - rss / tss
    return rsquared


## Cross-validation

### Target regressor vs. baseline regressor with _word frequency_, _length_, and _position_

In [5]:
for LM in LM_LIST:
    LM_ = LM.replace('-', '_')
    for N_SAMPLES in N_SAMPLES_LIST:
        for N_TOKENS in N_TOKENS_LIST:
            
            # Define exact metric names for the current LM, N_SAMPLES, N_TOKENS (and TEMPERATURE)
            EXACT_METRIC_NAMES = []
            for exact_metric in EXACT_METRICS:
                EXACT_METRIC_NAMES.append(f"{exact_metric}_{LM_}")
                EXACT_METRIC_NAMES.append(f"{exact_metric}_{LM_}_prev")
                EXACT_METRIC_NAMES.append(f"{exact_metric}_{LM_}_prev_prev")
            # Define sampling metric names 
            SAMPLING_METRIC_NAMES = []
            for sampling_metric in SAMPLING_METRICS:
                SAMPLING_METRIC_NAMES.append(f"{sampling_metric}_{LM_}_{N_SAMPLES}_{N_TOKENS}_{str(TEMPERATURE).replace('.', '_')}")
                SAMPLING_METRIC_NAMES.append(f"{sampling_metric}_{LM_}_{N_SAMPLES}_{N_TOKENS}_{str(TEMPERATURE).replace('.', '_')}_prev")
                SAMPLING_METRIC_NAMES.append(f"{sampling_metric}_{LM_}_{N_SAMPLES}_{N_TOKENS}_{str(TEMPERATURE).replace('.', '_')}_prev_prev")
            # These are all metrics that will be used in the regression models
            METRICS = EXACT_METRICS + SAMPLING_METRICS
            METRIC_NAMES = EXACT_METRIC_NAMES + SAMPLING_METRIC_NAMES

            # Data normalization
            scaler = MinMaxScaler()
            data_norm = data.copy()
            data_norm[METRIC_NAMES] = scaler.fit_transform(data[METRIC_NAMES])
            
            
            # Begin cross-validation
            results = []
            
            for predicted_var in tqdm(ALL_PREDICTED_VARIABLES):

                # For reading times, include values at t-1 and t-2 (spillover effects)
                if predicted_var in RT:
                    baseline_predictors = BASELINE_PREDICTORS_RT
                else:
                    baseline_predictors = BASELINE_PREDICTORS

                df_tmp = data_norm[[predicted_var] + baseline_predictors + METRIC_NAMES]

                # N_SEEDS cross-validation runs
                for seed in range(N_SEEDS):
                    kf = KFold(n_splits=N_FOLDS, random_state=seed, shuffle=True)
                    kf.get_n_splits(df_tmp) 

                    # N_FOLDS cross-validation
                    for fold, (train_indices, test_indices) in enumerate(kf.split(df_tmp)):

                        # Split data into training and test sets
                        df_tmp_fold = df_tmp.iloc[train_indices]
                        df_tmp_fold_test = df_tmp.iloc[test_indices]

                        # Fit baseline regressor
                        baseline_regressor = smf.ols(
                            formula=f'{predicted_var} ~ {" + ".join(baseline_predictors)}', 
                            data=df_tmp_fold
                        ).fit()

                        baseline_rsquared_test = compute_rsquared(baseline_regressor, df_tmp_fold_test, predicted_var)

                        # Target regressors
                        for metric in METRICS:
                            # Exact metrics (no MC sampling)
                            if metric in EXACT_METRICS:
                                if predicted_var in RT:
                                    # For reading times, include values at t-1 and t-2 (spillover effects)
                                    predictor = " + ".join([f"{metric.replace('-', '_')}_{LM_}{timestep}" for timestep in ["", "_prev", "_prev_prev"]])
                                else:
                                    predictor = f"{metric.replace('-', '_')}_{LM_}"
                            # Sampling metrics (MC sampling)
                            elif metric in SAMPLING_METRICS:
                                if predicted_var in RT:
                                    # For reading times, include values at t-1 and t-2 (spillover effects)
                                    predictor = " + ".join([f"{metric.replace('-', '_')}_{LM_}_{N_SAMPLES}_{N_TOKENS}_{str(TEMPERATURE).replace('.', '_')}{timestep}" for timestep in ["", "_prev", "_prev_prev"]])
                                else:
                                    predictor = f"{metric.replace('-', '_')}_{LM_}_{N_SAMPLES}_{N_TOKENS}_{str(TEMPERATURE).replace('.', '_')}"
                            else:
                                raise ValueError(f"Unknown metric: {metric}")
                            
                            # Fit target regressor
                            target_regressor = smf.ols(
                                formula=f'{predicted_var} ~ {" + ".join(baseline_predictors)} + {predictor}', 
                                data=df_tmp_fold
                            ).fit()

                            # Compute R-squared on test set
                            target_rsquared_test = compute_rsquared(target_regressor, df_tmp_fold_test, predicted_var)

                            results.append({
                                "y": predicted_var, 
                                "metric": metric,
                                "model": LM, 
                                "nsamples": "" if metric in EXACT_METRICS else N_SAMPLES,
                                "ntokens": "" if metric in EXACT_METRICS else N_TOKENS,
                                "temperature": "" if metric in EXACT_METRICS else TEMPERATURE,
                                "fold": f"{seed}_{fold}",
                                "rsquared_train": target_regressor.rsquared, 
                                "aic_train": target_regressor.aic,
                                "bic_train": target_regressor.bic,
                                "delta_rsquared_train": target_regressor.rsquared - baseline_regressor.rsquared, 
                                "delta_aic_train": target_regressor.aic - baseline_regressor.aic,
                                "delta_bic_train": target_regressor.bic - baseline_regressor.bic,
                                "rsquared_test": target_rsquared_test,
                                "delta_rsquared_test": target_rsquared_test - baseline_rsquared_test
                            })
                
            results_df = pd.DataFrame(results)

            results_df.to_csv(
                f"{RESULTS_FOLDER}/control_baseline_{LM_}_temp{str(TEMPERATURE).replace('.', '_')}_{N_SAMPLES}samples_{N_TOKENS}tokens_{N_SEEDS}x{N_FOLDS}fold.csv",
                index=False
            )


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

...

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

### Target regressor vs. baseline regressor with _surprisal_, _entropy_, _word frequency_, _length_, and _position_

In [6]:
for LM in LM_LIST:
    LM_ = LM.replace('-', '_')
    for N_SAMPLES in N_SAMPLES_LIST:
        for N_TOKENS in N_TOKENS_LIST:
            
            # Define exact metric names for the current LM, N_SAMPLES, N_TOKENS (and TEMPERATURE)
            EXACT_METRIC_NAMES = []
            for exact_metric in EXACT_METRICS:
                EXACT_METRIC_NAMES.append(f"{exact_metric}_{LM_}")
                EXACT_METRIC_NAMES.append(f"{exact_metric}_{LM_}_prev")
                EXACT_METRIC_NAMES.append(f"{exact_metric}_{LM_}_prev_prev")
            # Define sampling metric names 
            SAMPLING_METRIC_NAMES = []
            for sampling_metric in SAMPLING_METRICS:
                SAMPLING_METRIC_NAMES.append(f"{sampling_metric}_{LM_}_{N_SAMPLES}_{N_TOKENS}_{str(TEMPERATURE).replace('.', '_')}")
                SAMPLING_METRIC_NAMES.append(f"{sampling_metric}_{LM_}_{N_SAMPLES}_{N_TOKENS}_{str(TEMPERATURE).replace('.', '_')}_prev")
                SAMPLING_METRIC_NAMES.append(f"{sampling_metric}_{LM_}_{N_SAMPLES}_{N_TOKENS}_{str(TEMPERATURE).replace('.', '_')}_prev_prev")
            # These are all metrics that will be used in the regression models
            METRICS = EXACT_METRICS + SAMPLING_METRICS
            METRIC_NAMES = EXACT_METRIC_NAMES + SAMPLING_METRIC_NAMES

            # Data normalization
            scaler = MinMaxScaler()
            data_norm = data.copy()
            data_norm[METRIC_NAMES] = scaler.fit_transform(data[METRIC_NAMES])
            
            
            # Begin cross-validation
            results = []
            
            for predicted_var in tqdm(ALL_PREDICTED_VARIABLES):

                # For reading times, include values at t-1 and t-2 (spillover effects)
                if predicted_var in RT:
                    baseline_predictors = BASELINE_PREDICTORS_RT
                else:
                    baseline_predictors = BASELINE_PREDICTORS

                df_tmp = data_norm[[predicted_var] + baseline_predictors + METRIC_NAMES]  

                # N_SEEDS cross-validation runs
                for seed in range(N_SEEDS):

                    kf = KFold(n_splits=N_FOLDS, random_state=seed, shuffle=True)
                    kf.get_n_splits(df_tmp) 

                    # N_FOLDS cross-validation
                    for fold, (train_indices, test_indices) in enumerate(kf.split(df_tmp)):

                        # Split data into training and test sets
                        df_tmp_fold = df_tmp.iloc[train_indices]
                        df_tmp_fold_test = df_tmp.iloc[test_indices]

                        # Baseline predictors include surprisal and entropy
                        if predicted_var in RT:
                            # For reading times, include values at t-1 and t-2 (spillover effects)
                            formula = f'{predicted_var} ~ {" + ".join(baseline_predictors)} + {" + ".join([f"entropy_{LM_}{timestep}" for timestep in ["", "_prev", "_prev_prev"]])} + {" + ".join([f"surprisal_{LM_}{timestep}" for timestep in ["", "_prev", "_prev_prev"]])}'
                        else:
                            formula = f'{predicted_var} ~ {" + ".join(baseline_predictors)} + entropy_{LM_} + surprisal_{LM_}'
                        
                        # Fit baseline regressor
                        baseline_regressor = smf.ols(
                            formula=formula,
                            data=df_tmp_fold
                        ).fit()
                        
                        # Compute baseline regressor's R-squared on test set
                        baseline_rsquared_test = compute_rsquared(baseline_regressor, df_tmp_fold_test, predicted_var)

                        # Target regressors with anticipatory metrics
                        for metric in ANTICIPATORY:

                            # Collect target predictors
                            if metric == 'expected_prob':  # this is the only anticipatory metric that does not depend on sampling
                                if predicted_var in RT:
                                    # For reading times, include values at t-1 and t-2 (spillover effects)
                                    predictor = " + ".join([f"{metric}_{LM_}{timestep}" for timestep in ["", "_prev", "_prev_prev"]])
                                else:
                                    predictor = f"{metric}_{LM_}"
                            else:
                                if predicted_var in RT:
                                    # For reading times, include values at t-1 and t-2 (spillover effects)
                                    predictor = " + ".join([f"{metric}_{LM_}_{N_SAMPLES}_{N_TOKENS}_{str(TEMPERATURE).replace('.', '_')}{timestep}" for timestep in ["", "_prev", "_prev_prev"]])
                                else:
                                    predictor = f"{metric}_{LM_}_{N_SAMPLES}_{N_TOKENS}_{str(TEMPERATURE).replace('.', '_')}"

                            # Collect all predictors: here, we substitute entropy with the target anticipatory metric
                            if predicted_var in RT:
                                # For reading times, include values at t-1 and t-2 (spillover effects)
                                formula = f'{predicted_var} ~ {" + ".join(baseline_predictors)} + {predictor} + {" + ".join([f"surprisal_{LM_}{timestep}" for timestep in ["", "_prev", "_prev_prev"]])}'
                            else:
                                formula = f'{predicted_var} ~ {" + ".join(baseline_predictors)} + {predictor} + surprisal_{LM_}'
                            
                            # Fit target regressor with anticipatory metric
                            target_regressor_anticipatory = smf.ols(
                                formula=formula,
                                data=df_tmp_fold
                            ).fit()

                            # Compute R-squared on test set
                            anticip_rsquared_test = compute_rsquared(target_regressor_anticipatory, df_tmp_fold_test, predicted_var)

                            results.append({
                                "y": predicted_var, 
                                "metric": metric,
                                "model": LM,
                                "nsamples": "" if metric == 'expected_prob' else N_SAMPLES,
                                "ntokens": "" if metric == 'expected_prob' else N_TOKENS,
                                "temperature": "" if metric == 'expected_prob' else TEMPERATURE,
                                "fold": f"{seed}_{fold}", 
                                "rsquared_train": target_regressor_anticipatory.rsquared,
                                "aic_train": target_regressor_anticipatory.aic,
                                "bic_train": target_regressor_anticipatory.bic,
                                "delta_rsquared_train": target_regressor_anticipatory.rsquared - baseline_regressor.rsquared, 
                                "delta_aic_train": target_regressor_anticipatory.aic - baseline_regressor.aic,
                                "delta_bic_train": target_regressor_anticipatory.bic - baseline_regressor.bic,
                                "rsquared_test": anticip_rsquared_test,
                                "delta_rsquared_test": anticip_rsquared_test - baseline_rsquared_test,
                            })


                        # Target regressors with responsive metrics
                        for metric in RESPONSIVE:

                            # Collect target predictors
                            if metric == 'prob':  # this is the responsive metric that does not depend on sampling
                                if predicted_var in RT:
                                    # For reading times, include values at t-1 and t-2 (spillover effects)
                                    predictor = " + ".join([f"{metric}_{LM_}{timestep}" for timestep in ["", "_prev", "_prev_prev"]])
                                else:
                                    predictor = f"{metric}_{LM_}"
                            else:
                                if predicted_var in RT:
                                    # For reading times, include values at t-1 and t-2 (spillover effects)
                                    predictor = " + ".join([f"{metric}_{LM_}_{N_SAMPLES}_{N_TOKENS}_{str(TEMPERATURE).replace('.', '_')}{timestep}" for timestep in ["", "_prev", "_prev_prev"]])
                                else:
                                    predictor = f"{metric}_{LM_}_{N_SAMPLES}_{N_TOKENS}_{str(TEMPERATURE).replace('.', '_')}"

                            # Collect all predictors: here, we substitute surprisal with the target responsive metric
                            if predicted_var in RT:
                                # For reading times, include values at t-1 and t-2 (spillover effects)
                                formula = f'{predicted_var} ~ {" + ".join(baseline_predictors)} + {" + ".join([f"entropy_{LM_}{timestep}" for timestep in ["", "_prev", "_prev_prev"]])} + {predictor}'
                            else:
                                formula = f'{predicted_var} ~ {" + ".join(baseline_predictors)} + entropy_{LM_} + {predictor}'

                            target_regressor_responsive = smf.ols(
                                formula=formula,
                                data=df_tmp_fold
                            ).fit()

                            resp_rsquared_test = compute_rsquared(target_regressor_responsive, df_tmp_fold_test, predicted_var)

                            results.append({
                                "y": predicted_var, 
                                "metric": metric,
                                "model": LM,
                                "nsamples": "" if metric == 'prob' else N_SAMPLES,
                                "ntokens": "" if metric == 'prob' else N_TOKENS,
                                "temperature": "" if metric == 'prob' else TEMPERATURE,
                                "fold": f"{seed}_{fold}", 
                                "rsquared_train": target_regressor_responsive.rsquared,
                                "aic_train": target_regressor_responsive.aic,
                                "bic_train": target_regressor_responsive.bic,
                                "delta_rsquared_train": target_regressor_responsive.rsquared - baseline_regressor.rsquared,
                                "delta_aic_train": target_regressor_responsive.aic - baseline_regressor.aic,
                                "delta_bic_train": target_regressor_responsive.bic - baseline_regressor.bic,
                                "rsquared_test": resp_rsquared_test,
                                "delta_rsquared_test": resp_rsquared_test - baseline_rsquared_test
                            })

                
            results = pd.DataFrame(results)

            results.to_csv(
                f"{RESULTS_FOLDER}/surprisal_entropy_baseline_{LM_}_temp{str(TEMPERATURE).replace('.', '_')}_{N_SAMPLES}samples_{N_TOKENS}tokens_{N_SEEDS}x{N_FOLDS}fold.csv",
                index=False
            )


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

...

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]